In [212]:
import urllib2
from bs4 import BeautifulSoup
import bs4,time
import string
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import requests
from lxml import html

In [96]:
url = 'https://www.basketball-reference.com/teams/CHI/1987.html'
page = urllib2.urlopen(url)

In [97]:
soup = BeautifulSoup(page,'html5lib')

In [99]:
type(soup.body.div.contents[0])==bs4.element.NavigableString

True

In [100]:
type(soup.body.div.contents[1])

bs4.element.Tag

In [101]:
for n,i in enumerate(soup.body.div.children):
    if type(i)==bs4.element.Tag:
        print i['id'],n

header 1
info 4
srcom 6
inner_nav 9
content 12
footer 15


In [102]:
for n,i in enumerate(soup.body.div.contents[12].children):
    if type(i)==bs4.element.Tag:
        if 'id' in i.attrs:
            print i['id'],n

timeline_results 3
all_roster 5
all_assistant_coaches 11
all_team_and_opponent 13
all_team_misc 19
all_per_game 21
all_totals 23
all_per_minute 25
all_per_poss 27
all_advanced 29
all_playoffs_totals 31
all_playoffs_per_game 33
all_playoffs_per_minute 35
all_playoffs_per_poss 37
all_playoffs_advanced 39
bottom_nav 45


In [128]:
for n,i in enumerate(soup.body.div.contents[12].contents[39].children):
    print type(i),n

<class 'bs4.element.NavigableString'> 0
<class 'bs4.element.Tag'> 1
<class 'bs4.element.NavigableString'> 2
<class 'bs4.element.Tag'> 3
<class 'bs4.element.NavigableString'> 4
<class 'bs4.element.Comment'> 5
<class 'bs4.element.NavigableString'> 6


In [110]:
page = requests.get(url)

In [111]:
tree = html.fromstring(page.content)

In [161]:
def player_info():
    '''
    This function web scrapes basketball-reference and extracts player's info.
    '''
    players = []
    base_url = 'http://www.basketball-reference.com/players/'

    # get player tables from alphabetical list pages
    for letter in string.lowercase:
        page_request = requests.get(base_url + letter)
        soup = BeautifulSoup(page_request.text,"lxml")
        # find table in soup
        table = soup.find('table')

        if table:
            table_body = table.find('tbody')

            # loop over list of players
            for row in table_body.findAll('tr'):

                # get name and url
                player_url = row.find('a')
                player_names = player_url.text
                player_pages = player_url['href']

                # get some player's info from table
                cells = row.findAll('td')
                active_from = int(cells[0].text)
                active_to = int(cells[1].text)
                position = cells[2].text
                height = cells[3].text
                weight = cells[4].text
                birth_date = cells[5].text
                college = cells[6].text    

                # create entry
                player_entry = {'url': player_pages,
                                'name': player_names,
                                'active_from': active_from,
                                'active_to': active_to,
                                'position': position,
                                'college': college,
                                'height': height,
                                'weight': weight,
                                'birth_date': birth_date}

                # append player dictionary
                players.append(player_entry)
                
    return pd.DataFrame(players)

In [162]:
players_general_info = player_info() # call function that scrapes general info
players_general_info.head() # preview

,active_from,active_to,birth_date,college,height,name,position,url,weight
0,1991,1995,"June 24, 1968",Duke University,6-10,Alaa Abdelnaby,F-C,/players/a/abdelal01.html,240
1,1969,1978,"April 7, 1946",Iowa State University,6-9,Zaid Abdul-Aziz,C-F,/players/a/abdulza01.html,235
2,1970,1989,"April 16, 1947","University of California, Los Angeles",7-2,Kareem Abdul-Jabbar,C,/players/a/abdulka01.html,225
3,1991,2001,"March 9, 1969",Louisiana State University,6-1,Mahmoud Abdul-Rauf,G,/players/a/abdulma02.html,162
4,1998,2003,"November 3, 1974","University of Michigan, San Jose State University",6-6,Tariq Abdul-Wahad,F,/players/a/abdulta01.html,223


In [164]:
players_general_info.shape

(4656, 9)

In [165]:
validPlayersGeneralInfo = players_general_info[(players_general_info.active_to>1976) & 
                     ((players_general_info.active_to-players_general_info.active_from)>3)]
print validPlayersGeneralInfo.shape

(1652, 9)


In [201]:
pl_i = 631
print validPlayersGeneralInfo.url.values.tolist()[pl_i]

/players/h/harvean01.html


In [202]:
page_request = requests.get('http://www.basketball-reference.com' + validPlayersGeneralInfo.url.values.tolist()[pl_i])

In [208]:
np.random.choice(range(100),10).tolist()

[35, 63, 8, 97, 28, 50, 43, 84, 55, 57]

In [251]:
for player_idx in np.random.choice(range(validPlayersGeneralInfo.shape[0]),10).tolist():
    print player_idx
    page_request = requests.get('http://www.basketball-reference.com' + validPlayersGeneralInfo.url.values.tolist()[player_idx])
    soup = BeautifulSoup(page_request.text,"lxml")
    print extractData(soup).keys()
    time.sleep(np.random.randint(25))
    

816
['all_per_game', 'all_per_minute', 'all_per_poss', 'all_advanced', 'all_playoffs_per_game', 'all_playoffs_per_minute', 'all_playoffs_per_poss', 'all_playoffs_advanced']
['all_playoffs_per_poss', 'all_advanced', 'all_playoffs_advanced', 'all_per_game', 'all_playoffs_per_minute', 'all_per_poss', 'all_per_minute', 'all_playoffs_per_game']
218
['all_per_game', 'all_per_minute', 'all_per_poss', 'all_advanced', 'all_playoffs_per_game', 'all_playoffs_per_minute', 'all_playoffs_per_poss', 'all_playoffs_advanced']
['all_playoffs_per_poss', 'all_advanced', 'all_playoffs_advanced', 'all_per_game', 'all_playoffs_per_minute', 'all_per_poss', 'all_per_minute', 'all_playoffs_per_game']
655
['all_per_game', 'all_totals', 'all_per_minute', 'all_per_poss', 'all_shooting', 'all_year-and-career-highs', 'all_playoffs_per_game', 'all_playoffs_totals']
['all_per_poss', 'all_per_minute', 'all_playoffs_per_game', 'all_per_game']
1313
['all_per_game', 'all_totals', 'all_per_minute', 'all_per_poss', 'all_sho

In [203]:
soup = BeautifulSoup(page_request.text,"lxml")

In [169]:
for n,i in enumerate(soup.body.div.contents[13].contents[3].contents):
    print type(i),n
    if type(i)==bs4.element.Tag:
        if 'id' in i.attrs:
            print i['id'],n

<class 'bs4.element.NavigableString'> 0
<class 'bs4.element.Tag'> 1
<class 'bs4.element.NavigableString'> 2
<class 'bs4.element.Tag'> 3
<class 'bs4.element.NavigableString'> 4


In [249]:
## how to deal with did not play seasons (MJ)
## injured players

treeDict = {'all_per_game':[3,3,1,1,6,'odd'], 
            'all_per_minute':[13,5,"str",0,1,0,1,1,6,'all'],
            "all_per_poss":[19,5,"str",0,1,0,1,1,6,'all'],
            "all_advanced":[21,5,"str",0,1,0,1,1,6,'all'],
            "all_playoffs_per_game":[29,5,"str",0,1,0,1,1,6,'all'],
            "all_playoffs_per_minute":[33,5,"str",0,1,0,1,1,6,'all'],
            "all_playoffs_per_poss":[35,5,"str",0,1,0,1,1,6,'all'],
            "all_playoffs_advanced":[37,5,"str",0,1,0,1,1,6,'all']}

def traverseTree(obj,tr):
    if len(tr)==1:
        return obj,tr[0]
    elif type(tr[0])==str:
        converted = str(obj).replace('\n','').replace('\t','')
        return traverseTree(BeautifulSoup(converted,"html5lib"),tr[1:])
    else:
        return traverseTree(obj.contents[tr[0]],tr[1:])
    
def extractTableData(table,playerData,k,tableType):
    if len(table.contents)>1:
        if tableType=='odd':
            odds = range(1,len(table.contents),2)
            rows = []
            for o in odds:
                tableRow = table.contents[o]
                row = {}
                for i,c in enumerate(tableRow.contents):
                    try:
                        if len(c.contents)>0:
                            row[c['data-stat']] = c.contents[0]
                            ## must add stripping out contents by type
                    except:
                        continue
                rows.append(row)
            playerData[k] = rows
        elif tableType=='all':
            rows = []
            for j in range(len(table.contents))[:-1]:
                row = {}
                for i,c in enumerate(table.contents[j]):
                    try:
                        if len(c.contents)>0:
                            row[c['data-stat']] = c.contents[0]
                    except:
                        continue
                rows.append(row)
            playerData[k] = rows
    else: # deal with strings with only one year of playing time?...
        row = {}
        

def extractData(ppg):
    relevantSets = [3,13,19,21,29,33,35,37,]
    relevantSets = [r for r in relevantSets if type(ppg.body.div.contents[13].contents[r])!=bs4.element.Comment]
    playerData = {}
    print [ppg.body.div.contents[13].contents[k]['id'] for k in relevantSets]
    for r in relevantSets:
        key = ppg.body.div.contents[13].contents[r]['id']
        if key in treeDict:
            dataTable,tableType = traverseTree(ppg.body.div.contents[13],treeDict[key])
            extractTableData(dataTable,playerData,key,tableType)
    return playerData

In [250]:
extractData(soup).keys()
#pd.DataFrame(extractData(soup)['all_per_minute'])

['all_per_game', 'all_per_minute', 'all_per_poss', 'all_advanced', 'all_all_college_stats', 'all_9351239279']


['all_per_poss', 'all_per_minute', 'all_advanced', 'all_per_game']

In [248]:
type(soup.body.div.contents[13].contents[35])==bs4.element.Comment

True

In [217]:
soup.body.div.contents[13].contents[3].contents[3].contents[1].contents[1].contents[6].contents
#3,3,1,1,6,

[u'\n',
 <tr class="full_table" id="per_game.1981"><th class="left " data-stat="season" scope="row"><a href="/players/n/nattke01/gamelog/1981/">1980-81</a></th><td class="center " data-stat="age">22</td><td class="left " data-stat="team_id"><a href="/teams/IND/1981.html">IND</a></td><td class="left " data-stat="lg_id"><a href="/leagues/NBA_1981.html">NBA</a></td><td class="center " data-stat="pos">SG</td><td class="right " data-stat="g">19</td><td class="right iz" data-stat="gs"></td><td class="right " data-stat="mp_per_g">7.8</td><td class="right " data-stat="fg_per_g">1.3</td><td class="right " data-stat="fga_per_g">4.1</td><td class="right " data-stat="fg_pct">.325</td><td class="right " data-stat="fg3_per_g">0.1</td><td class="right " data-stat="fg3a_per_g">0.4</td><td class="right " data-stat="fg3_pct">.250</td><td class="right " data-stat="fg2_per_g">1.2</td><td class="right " data-stat="fg2a_per_g">3.6</td><td class="right " data-stat="fg2_pct">.333</td><td class="right " data-s

In [173]:
sample = str(soup.body.div.contents[13].contents[29].contents[5]).replace('\n','').replace('\t','')
# print sample
sample = BeautifulSoup(sample,"html5lib")
sample.contents[0].contents[1].contents[0].contents[1].contents[1].contents[6].contents

# for n,i in enumerate(soup.body.div.contents[13].contents[13].contents):
#     print type(i),n
#     if type(i)==bs4.element.Tag:
#         if 'id' in i.attrs:
#             print i['id'],n

[<tr class="bold"><th class="left " data-stat="player" scope="row">Jimmy Oliver</th><td class="right iz" csk="100" data-stat="sim_score"></td><td class="right " data-stat="year1">0.2</td><td class="right " data-stat="year2">0.2</td><td class="right " data-stat="year3">0.0</td><td class="right " data-stat="year4">0.0</td><td class="right " data-stat="year5">-0.1</td></tr>,
 <tr><th class="left " data-append-csv="pricemi01" data-stat="player" scope="row"><a href="/players/p/pricemi01.html">Mike Price</a></th><td class="right " data-stat="sim_score">78.8</td><td class="right " data-stat="year1">0.2</td><td class="right " data-stat="year2">0.2</td><td class="right " data-stat="year3">0.1</td><td class="right iz" data-stat="year4"></td><td class="right iz" data-stat="year5"></td></tr>,
 <tr><th class="left " data-append-csv="storeaw01" data-stat="player" scope="row"><a href="/players/s/storeaw01.html">Awvee Storey</a></th><td class="right " data-stat="sim_score">51.0</td><td class="right " 

In [231]:
len(soup.body.div.contents[13].contents)

45

In [88]:
for n,i in enumerate(soup.body.div.contents[13].contents):
    if type(i)==bs4.element.Tag:
        if 'id' in i.attrs:
            print i['id'],n

all_per_game 3
all_4721804306 5
all_totals 11
all_per_minute 13
all_per_poss 19
all_advanced 21
all_year-and-career-highs 27
all_playoffs_per_game 29
all_playoffs_totals 31
all_playoffs_per_minute 33
all_playoffs_per_poss 35
all_playoffs_advanced 37
all_year-and-career-highs-po 39
all_all_star 41
all_sim_thru 43
all_sim_career 45
all_all_college_stats 47
all_leaderboard 49
all_transactions 51
all_5448669928 53
bottom_nav 59


In [91]:
soup.body.div.contents[13].contents[3].contents[3].contents[1].contents[1].contents[6].contents
#3,3,1,1,6

[u'\n',
 <tr class="full_table" id="per_game.1972"><th class="left " data-stat="season" scope="row"><a href="/players/m/mcginge01/gamelog/1972/aba/">1971-72</a></th><td class="center " data-stat="age">21</td><td class="left " data-stat="team_id"><a href="/teams/INA/1972.html">INA</a></td><td class="left " data-stat="lg_id"><a href="/leagues/ABA_1972.html">ABA</a></td><td class="center " data-stat="pos">SF</td><td class="right " data-stat="g">73</td><td class="right iz" data-stat="gs"></td><td class="right " data-stat="mp_per_g">29.8</td><td class="right " data-stat="fg_per_g">6.4</td><td class="right " data-stat="fga_per_g">13.7</td><td class="right " data-stat="fg_pct">.465</td><td class="right " data-stat="fg3_per_g">0.1</td><td class="right " data-stat="fg3a_per_g">0.5</td><td class="right " data-stat="fg3_pct">.158</td><td class="right " data-stat="fg2_per_g">6.3</td><td class="right " data-stat="fg2a_per_g">13.2</td><td class="right " data-stat="fg2_pct">.478</td><td class="right 

In [92]:
for c in soup.body.div.contents[13].contents[3].contents[3].contents[1].contents[1].contents[6].contents[1].contents:
    print c['data-stat'],len(c.contents)

season 1
age 1
team_id 1
lg_id 1
pos 1
g 1
gs 0
mp_per_g 1
fg_per_g 1
fga_per_g 1
fg_pct 1
fg3_per_g 1
fg3a_per_g 1
fg3_pct 1
fg2_per_g 1
fg2a_per_g 1
fg2_pct 1
efg_pct 1
ft_per_g 1
fta_per_g 1
ft_pct 1
orb_per_g 1
drb_per_g 1
trb_per_g 1
ast_per_g 1
stl_per_g 0
blk_per_g 0
tov_per_g 1
pf_per_g 1
pts_per_g 1


In [194]:
for n,i in enumerate(soup.body.div.contents[13].contents[3].contents[3].contents[1].contents[1].contents[6].contents):
    print type(i),n

<class 'bs4.element.NavigableString'> 0
<class 'bs4.element.Tag'> 1
<class 'bs4.element.NavigableString'> 2
<class 'bs4.element.Tag'> 3
<class 'bs4.element.NavigableString'> 4
<class 'bs4.element.Tag'> 5
<class 'bs4.element.NavigableString'> 6
<class 'bs4.element.Tag'> 7
<class 'bs4.element.NavigableString'> 8
<class 'bs4.element.Tag'> 9
<class 'bs4.element.NavigableString'> 10
<class 'bs4.element.Tag'> 11
<class 'bs4.element.NavigableString'> 12
<class 'bs4.element.Tag'> 13
<class 'bs4.element.NavigableString'> 14
<class 'bs4.element.Tag'> 15
<class 'bs4.element.NavigableString'> 16
<class 'bs4.element.Tag'> 17
<class 'bs4.element.NavigableString'> 18
<class 'bs4.element.Tag'> 19
<class 'bs4.element.NavigableString'> 20
<class 'bs4.element.Tag'> 21
<class 'bs4.element.NavigableString'> 22
<class 'bs4.element.Tag'> 23
<class 'bs4.element.NavigableString'> 24
<class 'bs4.element.Tag'> 25
<class 'bs4.element.NavigableString'> 26
<class 'bs4.element.Tag'> 27
<class 'bs4.element.NavigableS